# Creating a sales dashboard in Google Data Studio

## Goals

In this case, you will learn how to plan and build a visual dashboard in Google Data Studio. You will expand upon the skills you learned about choosing visualizations and presenting them in a compelling and clear way.

## Introduction

**Business Context.** You are a member of the business analytics team of *Simply Music*, a Florida-based musical instrument store that has been growing at a fast pace over the last few years. Management is keen to find out what the most popular items are and which locations are the most profitable, in order to adjust their marketing strategies and their stock logistics to increase sales and revenue. They also want to learn about the number of deliveries made outside Florida, since they are contemplating opening a new branch in another state.

**Business Problem.** You are tasked with creating a sales dashboard for management to use. This dashboard must display sales, revenue, and profit over time, and allow for queries across product, location, and customer age. From your previous meetings with the management, you know that they are interested in having these three broad questions answered:

1. How have our sales, revenue and profits evolved over time?
1. What are the locations with the highest numbers of sales and profit?
1. What are the items with the highest numbers of sales, profit and revenue across: product description, product type, product line, location, customer age group and state of delivery?

**Analytical Context.** We have been given a sample dataset of 5000 sales records from Simply Music, corresponding to the years 2018 and 2019. They ask you to create the dashboard with this sample data, and after this is done and your dashboard receives the management's approval, your team will have to connect it to an online database to feed it the real time data that comes from the company's sales tracking software. The sample table and the database share the same structure, so you don't have to worry about database management or creation. You will plan your dashboard and deploy it using the free tool [Google Data Studio](https://datastudio.google.com/).

The case is structured as follows. You will:

1. Understand the business process that the dashboard has to represent.
1. Plan the dashboard layout and visualizations.
1. Lastly, you will create your dashboard using Google Data Studio.




In [ ]:
# We will use pandas to explore our data before uploading it to Google Data Studio (we won't modify the data)
import pandas as pd

## Understanding the business process

Data scientists are often required to extract valuable information from datasets without being given specific questions to answer. We usually address these cases with exploratory data analysis combined with domain knowledge. Other times, we can get the clients to articulate a limited set of questions. There is a third scenario, where the clients want to explore the data themselves in such a way that they can easily answer *ad hoc* queries that can't possibly be known in advance. This is much more open-ended and requires us to intimately understand the natural structure of the data and likely queries.

One way to address this need is to create dashboards that organize and display all relevant business information, as well as various filters that users might want to have, while being fully extendable if required in the future. When you create dashboards, you define a set of relevant variables and feed them into interactive visualizations that the users can then filter at will to explore the different relationships between the variables.

Dashboards work best when they mirror the business process that underlie the data. It is always advised that you study the greater business context so that you can put yourself in the shoes of your end users (in this case, management) and deliver a product that is user-friendly and pertinent.

### Specifics of <i>Simply Music</i>

Simply Music is a retailer, so it sells mostly to individual buyers who buy in small quantities. The customers are not restricted to a single state, and in fact many of them buy online from all over the country. There are three physical shops as well though, and they are all in the state of Florida.

These are the product lines that the store serves:

1. Musical instruments
    1. Acoustic pianos
    2. Digital pianos and keyboards
    3. Guitars (electric and acoustic)
    4. Woodwinds (flutes, oboes, bassoons, saxophones, etc.)
    5. Brass (trumpets, horns)
    6. Synthetics
    7. Strings (everything bowed: violins, celli, etc.)
    8. Percussion (mostly drumkits)
2. Accessories
    1. Strings (for guitars, violins, etc.)
    2. Pedals and amps (for electric guitars)
    3. Sheet music

Furthermore, these are the four selling channels that the company uses:

1. Physical store in Fort Lauderdale
2. Physical store in Tampa
3. Physical store in West Palm Beach
4. Online store

Each transaction is recorded at the time of purchasing, whether the purchase happens online or at a physical location. Accepted means of payment include cash and credit card. If the customer is not happy with their product, they can file a complaint within 10 days of the purchase.

## Looking at the data

The database team has given us access to a sample of 5,000 transactions and some other relevant data.

The first table is called `transactions`. Let's look at it using `pandas` and the `.head()` method:

In [ ]:
transactions = pd.read_csv('data/transactions.csv')
transactions.head()

There are several attributes in this table, namely:

* **TIME_KEY**: A key that refers to the year, month and day of the purchase. It's in YYMMDD format (e.g. `180122` links to January 22, 2018)
* **PRODUCT_KEY**: A number that uniquely identifies the product
* **UNITS_SOLD**: The units sold
* **DOLLARS_SOLD**: The total amount of dollars sold of this product. This variable is equal to the units sold times the unit price
* **CHANNEL**: Either `Fort Lauderdale`, `Tampa`, `West Palm Beach` or `Website`
* **CUSTOMER_KEY**: A number that uniquely identifies the customer
* **DESTINATION_STATE**: The state to which the customer wants their products delivered. Available only for online transactions

Additionally, we also have the `products` table:

In [ ]:
products = pd.read_csv('data/products.csv')
products.head()

The attributes are:

* **PRODUCT_KEY**: The unique identifier of the product
* **PRODUCT_LINE**: The product line (either "Musical instruments" or "Accessories")
* **PRODUCT_TYPE**: The type of the product (e.g. "Accoustic pianos", "Brass")
* **DESCRIPTION**: The name of the product (e.g. "Casio Digital Piano AP650")
* **COST**: How much the company estimates the item cost them (includes fixed and variable costs). This is unit cost
* **UNIT_PRICE**: The unit price

Let's also load in the `time` and `customers` tables:

In [ ]:
time = pd.read_csv('data/time.csv')
time.head()

The attributes of the `time` table are:

* **TIME_KEY**: The time key. It's in YYYYMMDD format (e.g. `20180122` links to January 22, 2018)
* **YEAR**: The year
* **MONTH**: The month (in MM format, so February becomes `02`)
* **DAY**: The day (in DD format, so the first becomes `01`)
* **WEEK_OF_YEAR**: The week of the year (the week of Jan 01 counts as week 1)
* **QUARTER**: The quarter to which the month belongs

Notice that we don't have data for hours or minutes.

In [ ]:
customers = pd.read_csv('data/customers.csv')
customers.head()

The fields of the `customers` table are:

* **CUSTOMER_KEY**: The unique identifier of the customer (can be a social security number or any other number)
* **CUSTOMER_NAME**: The customer's full name
* **CUSTOMER_AGE**: The age of your customer
* **CUSTOMER_AGE_GROUP**: The customer's age group

You might be wondering how we're going to pool all this data in order to create our dashboard. The good news is that Google Data Studio offers a really easy way to merge tables, so in many scenarios you don't have to worry too much about that. And the other good news is that we've pre-merged the data for you (the `simply_music.csv` dataset, that you can explore using `pd.read_csv('data/simply_music.csv)` if you like).

## Choosing the right visualizations

Let's keep in mind our problem statement: "The dashboard must display sales, revenue and profit across product type, location and customer age." What visualizations are most appropriate given this?

Google Data Studio offers a variety of visualizations, ranging from tables and plots to simple geographical maps. Besides most of the usual visualizations that you'll find in [Google Spreadsheets](https://support.google.com/docs/answer/190718?hl=en), there are some others that you can check [here](https://datastudio.google.com/visualization). We'll review several options to help us decide which ones suit our needs best.

You might recall that data can be *categorical* or *numerical*. In turn, numerical data can be of type *integer* or *float*. Data types are probably the key criterion to consider when it comes to choosing a visualization. Secondary criteria include ease of interpretation and the familiarity of your end user with a specific kind of plot.

### Categorical vs. numerical

When you need to plot a categorical variable against a numerical variable, your most likely choices are **bar charts** and **pie charts**, which we have already discussed in previous cases. A third option is using **gauges**. If you have variables whose values happen to lie inside the same given range, then you can plot them as gauge meters:

<img src="data/img/gauge.jpg" width="500"/>

Finally, if your categorical variable is geographical (states, countries), you can create **choropleth maps** (the numerical variable is represented as a hue):

<img src="data/img/choropleth.jpg" width="500"/>

A useful variant is using bubbles instead of tiles, where the size of the bubble (not the color) represents the magnitude of the value.

### Numerical vs. numerical

Here you can't go wrong with **scatterplots**. You can even add a third numerical variable or even a categorical variable into the mix, and create a **bubble chart**, in which the third numerical variable is represented as the size of the bubble and the categorical variable maps to a color code.

### Numerical over time

Here you might find **line plots** handy. You can add more **series** to the plot (i.e. more lines) to compare across different categorical variables over time. When we add more series, we tend to differentiate them using a different color for each one. Here's an example:

<img src="data/img/line_chart.jpg" width="500"/>

### Tables and scorecards

The last visualizations we'll cover aren't actually plots. You can always represent your data in tabular form, and that's sometimes more appropriate than a visual chart. The general principle is that you should keep tables simple and uncluttered. If that's not possible, then try some of the other visualizations explained above.

A common use case of tables is when you need to do a cross-tabulation (crosstab) of two categorical variables. Crosstabs, or **contingency tables**, as they are also called, count the number of elements of category A that also belong to category B, and present the results in a matrix. Like this, for instance:

| Type | Physical | Online |
| --- | --- | --- |
| Guitars | 40 | 60 |
| Acc. Pianos | 30 | 50 |

This table shows, for example, that the sold items belonging to "guitars" and "physical" (sold in a physical location) amount to 40. To learn more about contingency tables, check out [this article](https://en.wikipedia.org/wiki/Contingency_table).

You can display a contingency table as is in your dashboard if you like, but you can also slice it with a filter and then plot the relevant slice as a bar chart or a pie chart. Let's say you use a filter to slice out "online" (i.e. you only want to see sales from physical locations), so the resulting plot can be:

<img src="data/img/physical_crosstab_plot.jpg" width="500"/>

You can alternatively use a grouped bar chart (if you don't want to do the slicing), but grouped bar charts get difficult to read very quickly with more and more groupings, so they are really only recommended if the number of bars isn't too large.

In addition to these alternatives, you can also display single values using **scorecards**, like this:

<img src="data/img/scorecard.jpg" width="100"/>

### Applying our knowledge

Now that you know what visualizations go well with what kinds of data, it's time to choose which ones we'll use. Here's a table showing the variables that management requested to have in the dashboard, along with their data types:

| Variable | Type |
|-|-|
| Location | Categorical |
| State of delivery | Categorical |
| Time | Date |
| Customer age | Categorical (we'll use age group instead of simple ages) |
| Product type | Categorical |
| Product description | Categorical |
| Product line | Categorical |

These are the **dimensions** of our dashboards. Dimensions are the variables that we'll use to filter the data.

The numerical variables that our report will calculate and display are:

| Variable | Type |
|-|-|
| Revenue | Numeric (float) |
| Profit | Numeric (float) |
| Items sold | Numeric (integer) |
| Number of customers | Numeric (integer) |

These are called the **metrics** of our dashboard. You can say that metrics are numbers that are calculated and dimensions are filters that we use to determine which subsets of the data are used to compute those metrics.

<img src="data/img/metrics_and_dimensions.jpg" width="500"/>

The mathematical operations dashboards typically (but not exclusively) perform to create the metrics are these:

1. **Sum**: sums all the values of the rows of that column
2. **Count**: counts how many rows there are in the selected column
3. **Count distinct**: ounts how many non-duplicate rows there are in the selected column
4. **Average**: finds the arithmetic mean of all the values of the rows of that column
5. **Minimum**: finds the minumum of all the values of the rows of that column
6. **Maximum**: finds the maximum of all the values of the rows of that column

### Exercise 1

What mathematical operations are most appropriate to:

1. Calculate the total number of items sold
2. Calculate the total number of customers
3. Calculate total profit
4. Calculate total revenue

**Answer**. Shown below:

1. Calculate the total number of items sold: **Count**. You would count the number of rows in the `transactions` table
2. Calculate the total number of customers: **Count distinct**. You would count the number of unique customer identifiers in the `transactions` table. Since one single customer can make more than one purchase, you have to use count distinct to avoid counting them more than once
3. Calculate total profit: **Sum**. You would sum the profit earned from each transaction
4. Calculate total revenue: **Sum**. You would sum the revenue earned from each transaction in the `transactions` table

### Exercise 2

This is a table with all the possible pairwise combinations of dimensions. For each combination, it is possible to compute one or more metrics.  For instance, with "Location" and "Customer age group" we can report how many items were sold by location and age group, or how much profit each age bracket produces in each location, or even how many customers from each age group buy in that location.

Please fill in this table with suitable visualization options (just filling the upper half is enough, as the bottom half is functionally equivalent).

| var/var | Location | State of delivery | Customer age group | Product type | Product description | Product line |
|-|-|-|-|-|-|-|
| Location | * | <> | <> | <> | <> | <> |
| State of delivery | <> | * | <> | <> | <> | <> |
| Customer age group | <> | <> | * | <> | <> | <> |
| Product type | <> | <> | <> | * | <> | <> |
| Product description | <> | <> | <> | <> | * | <> |
| Product line | <> | <> | <> | <> | <> | * |

**Answer**. One possible solution is shown below:

| var/var | Location | State of delivery | Customer age group | Product type | Product description | Product line |
|-|-|-|-|-|-|-|
| Location | * | Choropleth | Cont. tab. or bar/col/pie | Cont. tab. or bar/col/pie | Cont. tab. or bar/col/pie | Cont. tab. or bar/col/pie |
| State of delivery | <> | * | Choropleth | Choropleth | Choropleth | Choropleth |
| Customer age group | <> | <> | * | Cont. tab. or bar/col/pie | Cont. tab. or bar/col/pie | Cont. tab. or bar/col/pie |
| Product type | <> | <> | <> | * | Cont. tab. or bar/col/pie | Cont. tab. or bar/col/pie |
| Product description | <> | <> | <> | <> | * | Cont. tab. or bar/col/pie |
| Product line | <> | <> | <> | <> | <> | * |

The choice between column, bar, or pie chart depends a lot on aesthetics, ease of interpretation, and purpose. As we mentioned earlier, pie charts are good when you want to represent relative proportions of a few key categories. However, they are not recommended if you have many categories, or if you care about the absolute level of some quantity.

### Exercise 3

Let's say we want to plot revenue, profit, and sales over time. What would be an appropriate visualization?

**Answer**. Since we're dealing with dates, the best choice would be line plots. You can apply filters to the line plots so that the data only includes what you need. For instance, you could filter by a given age group (e.g. 50 - 60 years) and state of delivery ("Michigan"), and the plotted data would be updated accordingly. However, the filters wouldn't change the plot type itself.

## Planning out the layout of our dashboard

The table above was just a list of pairwise combinations of variables, but having a bunch of visualizations with no logical order whatsoever won't be pleasing to the eye and, most importantly, won't convey any useful information.

When the end user opens the dashboard, they should see a clear, self-descriptive roadmap of what it has to offer. A good dashboard is like a restaurant menu that categorizes the meals, arranges them in hierarchies according to what the customer wants to read first, and provides only the information needed to make a decision. If the customer wants to learn more about a dish (e.g. the specific ingredients used or the name of the chef) they would have to ask the waiter or waitress. So, your dashboard should present the most relevant information first and show more detailed information only if the user asks for it.

Keep the following image in mind, it will help us as we move forward:

<img src="data/img/menu.jpg" width="500"/>

A good way to organise our dashboard is by creating **pages**. These are screens that show different aspects of your data, and the end user can switch between them by using a menu bar. Let's start with a basic set of pages - a "Bird's-Eye View" page (with an overview of the metrics) and a "Detailed View" page (which has more information and more filters).

Our dashboard will have three main layout components - a sidebar, a banner, and a body:

<img src="data/img/main_layout.jpg" width="500"/>

The sidebar will contain links to the pages. The banner will have the title of the page and optionally notes, date filters and scorecards. The page body will contain the visualizations.

### The "Bird's-Eye View" page layout

Recall the original business problem. The managers want to know:

1. How have our sales, revenue and profits evolved over time?
2. What are the locations with the highest numbers of sales and profit?
3. What are the items with the highest numbers of sales, profit and revenue across: product description, product type, product line, location, customer age group and state of delivery?

The third question has lots of dimensions, so we can put it in the "Detailed View" page. Let's put the first two questions in the "Bird's-Eye View" page.

We know that question 1 can be answered using line plots. Since revenue and profits are both measured in dollars, we will show both of them in a single line plot using colored series, like this:

![A line plot](data/img/first_question_line_plot.png "A line plot")

Sales are measured in number of items, so they should have a line plot of their own. But what about the second question?

### Exercise 4

What visualizations would be appropriate to answer the second question?

> 2. What are the locations with the highest numbers of sales and profit?

**Answer.**. We can use bar plots. And again, since profit and revenue are measured in a common unit, they can be displayed in the same chart, grouped. Because the number of bars would be small (there are only four locations), this shoudn't be too difficult to interpret.

It would also be nice to give our users a summary of the whole page in a small set of aggregations. We can use scorecards for that. Let's create a scorecard for "Total revenue", one for "Total profit" and one for "Total sales", since these are the main metrics the managers are interested in.

So, the layout of this page could be sketched like this:

<img src="data/img/birds_eye_layout.jpg" width="500"/>

As we said before, we are trying to make sure that the visualizations that are more important to the users appear first; that is, closer to the left-upper quadrant of the screen.

### The "Detailed View" page layout

Let's now move onto the third question. Since we want to know sales and total profit for each item, it makes sense to have a chart for sales (measured in number of items) and a chart for profit (measured in dollars). Items and dollars are both numeric (the former is integer, the latter is float). As items are categoical, bar charts would be fine. (Pie charts might not be easy to interpret because there are many products.)

If you recall the `products` table, you'll remember that each product has a line ("Musical instruments"), a type ("Guitars"), and a description ("Fender American Stratocaster"). To avoid having to create a page-level filter for each one of these variables, we can add them to a single "Items" chart as **drill-down** variables. Drill-down allows you to select one element of a chart and move to a lower level in the hierarchy. This GIF might be helpful to understand this concept than a mere description in words (this is a bar chart of sales by item, with drill-down by line, type and description):

![Drill down](data/img/drill_down.gif "Drill down")

So, we can have a bar chart like this and next to it another bar chart (with no drill-down this time) that shows total profit for each *item*. We want our drill-down in the first chart to filter the second chart, so that when someone drills down by "Guitars", the second chart only shows guitars. This can be done easily in Google Data Studio.

### Exercise 5

Now we have the two main charts of the "Detailed View" page. Let's add some secondary charts (which will be smaller in size to represent their auxiliary status) for the dimensions that our management wants to filter the data across. These include location (items sold in each location), customer age group (number of customers in each age bracket) and state of delivery (items delivered). Which three plots would go well with these dimensions?

**Answer.** One possible answer is:

* Location - pie chart. It would have only 4 slices (4 locations), so it will be easy to interpret
* Customer age group - bar chart
* State of delivery - choropleth (or geographic map with bubbles)

If we add some scorecards to display summaries and a date picker, our layout could be like this:

<img src="data/img/detailed_view_layout.jpg" width="500"/>

**Important:** It is key that each visualization is linked to the question it is trying to answer. For each visualization, there will be a little text box stating the question and some other information that the user might find crucial to understand the chart.

Now we're all set. Let's move on to builiding our dashboard.

## Creating our dashboard in Google Data Studio

### Loading our data

Let's open [Google Data Studio](https://datastudio.google.com). The first thing we have to do is upload our data. Go to "Create -> Data source":

![Create source](data/img/create_source.png "Create source")

Click "File Upload" and then "Click to Upload Files". Upload the `simply_music.csv` table:

![CSV upload](data/img/file_upload.png "CSV upload")

![Drag files](data/img/drag_files.png "Drag files")

After you have uploaded the table from your computer, edit the name of the connection (I used "Simply Music") in the upper right-hand corner of the screen, and then hit "Connect". Click on "Create Report":

![Create report](data/img/create_report.png "Create report")

A new tab will be created and in it you'll see a pop-up asking if you want to add this data to the report. Hit "Add to Report". You can name your report now if you like (I used "Simply Music 2018-2019 Report").

### Creating the pages

Before creating our pages, we first need to create the sidebar where they will live. Click on any blank space on the canvas, and then click on "Layout" in the "Theme and Layout" sidebar. Select "Left" under "Navigation Position":

![Navigation Left](data/img/navigation_left.png "Navigation Left")

Now go to "Page -> New Page". Remember that we have two pages - Bird's-Eye View and Detailed View. Now, if you click on "View" (the blue button on the top), you'll see your blank report with a sidebar and the pages there. You can rename your pages (while in Edit mode) by clicking on the page counter and then the three dots beside the page name:

![Rename Page](data/img/rename_page.png "Rename Page")

### The Bird's-Eye View page

If you check the data carefully, you'll see that there is no `PROFIT` column. Since this will be one of the main metrics of our dashboard, we need to create it. This is our definition of profit:

> PROFIT = DOLLARS_SOLD - TOTAL_COST

`TOTAL_COST` doesn't exist either, so we'll create it. The formula for this variable is:

> TOTAL_COST = UNITS_SOLD * COST

To create these variables, we add a new chart first. Click on "Add a chart" in the tools bar and select "Bar chart". This or something similar will appear on the canvas:

<img src="data/img/first_chart.png" width="500"/>

Then select the chart and go to "Data" in the right sidebar. At the bottom, you'll see the option to add a field - click on that. Name it `TOTAL_COST` and write the following in the formula box:

```
UNITS_SOLD*COST
```

Save. Now repeat the process, only this time name the new field `PROFIT` and put this in the formula box:

```
DOLLARS_SOLD - TOTAL_COST
```

Now we have our `PROFIT` column and are ready to create the first chart. This will be a line chart, according to the layout we planned.

So now let's select our chart, go to the top of the right sidebar and click on "Chart" and select a line chart from the palette. If the first chart you created was already a line chart, you can just keep working with it.

Charts in Google Data Studio have several important inputs:

* **Dimension**: The dimension that will be plotted (the categories)
* **Date range dimension**: The dimension that will be used as the time dimension. Google Data Studio will find it automatically for us, so we won't modify it. It is the `TIME_KEY` from our data
* **Metric**: The number that will be computed and plotted
* **Drill-down**: Whether the plot will have drill-down or not
* **Interactions (apply filter)**: This checkbox will be checked only if we want our plot to double both as a plot and as a page-level filter

### Exercise 6

Take a minute to find these options in the DATA panel.

--------

Let's now create our line chart. The dimension will be the date (since it is a time series plot) and the metrics will be revenue and profit. Drag and drop the `TIME_KEY` column into the "Dimension" box, and the `DOLLARS_SOLD` and the `PROFIT` columns into the "Metrics" box (note that `DOLLARS_SOLD` is the same as revenue).

Let's change the date granularity to "Month". Click on the small box to the left of `TIME_KEY` in the "Dimension" box and pick the granularity. You can change the name of the dimension as well if you like.

Make sure that the metrics are all set to "Sum" (remember Exercise 1) and change their names if needed.

Since we want our users to be able to filter the page by just clicking on the chart, we also check the "Apply filter" checkbox. Now add a text box above the chart with the question and some useful information, and your chart should be looking something like this:

![Line chart of Bird's-eye View](data/img/line_chart_birdseye.png "Line chart of Bird's-eye View")

### Exercise 7

Do the same for your sales line chart.

**Answer.** Your chart should look like this:

![Line chart 2 of Bird's-eye View](data/img/line_chart_sales_birdseye.png "Line chart 2 of Bird's-eye View")

### Adding the rest of the charts to the Bird's-Eye View page

Let's do the same for the rest of our charts (including the scorecards) and let's add the company logo as well (it is in the `data/img` folder as `logo.png`). To insert an image, go to "Insert -> Image" and then upload it from the button in the right sidebar.

The colors that go best with our logo are salmon and grey. Luckily, there's a theme preset in Google Data Studio with exactly those colors. Click anywhere in the canvas (though not on a chart) and select "Theme" in the right sidebar. The "Insight" theme looks nice, so click on it.

Finally, let's add the date picker. For the date picker, you go to "Add a Control" (in the toolbar) and then "Date Range Control". With it, the user will be able to pick a date range and all the charts in the page will be updated accordingly.

We add some more explanatory text, and our first page is done:

![First page](data/img/first_page.png "The first page")

### Exercise 8

Now do the same with the second page, the "Detailed View".

**Note 1:** We said we wanted to add drill-down to the first bar chart on this page. To achieve that, simply check the "Drill-down" checkbox under the "DATA" tab in the right sidebar while having the chart selected (don't forget to check "Apply filter" as well, because we want this chart to be a filter). Then add the drill-down dimensions, like this:

![How to drill down](data/img/how_to_drill_down.png "How to drill down")

**Note 2:** If you would also like to be able to filter the whole page by clicking on any of the bars/slices/bubbles on the chart, simply check "Apply filter" in each of the charts.

**Note 3:** For the geographic map, we recommend you use "Google Maps" (bubble map) as the chart type and not a choropleth ("Geomap"), since it is easier to set up. The settings for this chart should look like this:

![Google maps options](data/img/google_map_options.png "Google maps options")

**Answer**. Your second page should be similar to the below (we've tweaked the aesthetics just a bit, so don't worry if the look isn't exactly the same):

![Second page](data/img/second_page.png "The second page")

## The end result

Here's a GIF of what our dashboard looks like:

![The end result](data/img/end_result.gif "The end result")

Don't forget to share your report (the share link can be generated by clicking on the "Share" button). You can download your report in PDF format as well ("File -> Download as").

Your report is now ready to be presented to users. It will hopefully receive approval and be deployed using the live database. We won't cover how to replace the data source, but you can read about that [here](https://support.google.com/datastudio/answer/6350638?hl=en). It is usually a very straighforward process.

## Conclusions

In this case, we looked at different types of visualizations and studied what kinds of data they are best suited to. But we also saw that creating an analytical dashboard goes beyond this and involves thinking about the concrete business questions that the end users want to answer with the help of the dashboard. That is why we started by getting a grasp of the business process and the relevant dimensions and metrics, which in turn allowed us to move forward to the design of the layout and the choice of visualizations. Finally, we used Google Data Studio to build and publish the dashboard using advanced options like drill-downs and charts that doubled as page-level filters.

## Takeaways

Dashboard creation should always start with a sound understanding of the business process and the business questions. You also need to be familiar with the different types of visualizations and their strengths and weaknesses, and keep some basic design principles in mind as well. This allows you to build a dashboard that is functional but also not cluttered. Some key ideas you might want to keep in mind for the future are:

* Dashboards are like restaurant menus. They must be clear, straightforward and avoid overwhelming the user with too much information
* Your visualizations must always be organized hierarchically. The most important questions come first. Secondary questions can be placed more to the right or bottom, and be presented in a smaller size
* Each visualization must answer a business question, so it is always a good practice to label the chart or table with the question. You should also add explanatory text when you feel your users would appreciate it. However, if you find yourself adding too much text, then perhaps it is because your dashboard is not clear enough by itself and you should re-evaluate your layout or visualizations
* Drill-down and charts that double as page-level filters can save you a lot of space because you don't need to add more controls (like drop-down lists) to act as filters in the page. Not everything should be drillable, though, because the dashboard should be kept as simple and intuitive as possible

Google Data Studio is an excellent dashboarding tool, and its graphical user interface makes it one of the easiest to use. We encourage you to explore the many possibilities that it offers. This will help you expand your data visualization vocabulary and hone your dashboarding skills at the same time.

## Attribution

"American restaurant menu", Eric Fischer, April 14th, 2018, [Creative Commons Attribution 2.0 generic license](https://creativecommons.org/licenses/by/2.0/deed.en), https://commons.wikimedia.org/wiki/File:American_restaurant_menu_(30085586928).jpg#file

